In [43]:
from neo4j import GraphDatabase

class Neo4jGraphBuilder:
    def __init__(self, uri, user, password, database):
        self.driver = GraphDatabase.driver(uri, auth=(user, password), database=database)

    def close(self):
        self.driver.close()

    def create_node(self, concept):
        with self.driver.session() as session:
            session.run(
                "MERGE (c:Concept {name: $name})",
                name=concept
            )

    def create_relationship(self, node1, node2, relationship):
        with self.driver.session() as session:
            session.run(
                f"MATCH (a:Concept {{name: $node1}}), (b:Concept {{name: $node2}}) "
                f"MERGE (a)-[:{relationship}]->(b)",
                node1=node1,
                node2=node2
            )



In [49]:
# Exemple de fonction pour construire le graphe
def build_graph(concept_list):
    # Remplacez les valeurs par celles correspondant à votre configuration
    uri = "bolt://localhost:7687"  
    user = "neo4j"                
    password = "abdou12345;"  
    database = "database"  # Spécifiez le nom de la base de données ici
    
    # Instanciation de la classe de construction du graphe
    graph_builder = Neo4jGraphBuilder(uri, user, password, database)  # Remplacez par vos informations

    # Parcours de la liste de concepts pour créer des nœuds et des relations
    for concept in concept_list:
        node1 = concept.get("node_1")  # Extraction du nom du premier nœud
        node2 = concept.get("node_2")  # Extraction du nom du deuxième nœud
        relationship = concept.get("edge")  # Extraction de l'attribut edge
        
        # Création des nœuds
        graph_builder.create_node(node1)
        graph_builder.create_node(node2)

        # Créer une relation entre node1 et node2
        graph_builder.create_relationship(node1, node2, relationship)

    # Fermer la connexion à la base de données
    graph_builder.close()




In [50]:
# Utilisation de la fonction pour construire le graphe à partir de votre liste
concept_list = [  # Remplacez par votre liste de concepts
    {
        "node_1": "Project Management Institute",
        "node_2": "Practice Standard",
        "edge": "publishes"  # L'attribut edge est ici
    },
    {
        "node_1": "Practice Standard",
        "node_2": "Six Project Risk Management Processes",
        "edge": "outlines"  # L'attribut edge est ici
    },
    # Ajoutez d'autres concepts selon vos besoins
]

# Construire le graphe
build_graph(concept_list)